### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import psycopg2
import os

### Downloading the data to be analyzed

In [3]:
conn = psycopg2.connect(
    host="zip_data_postgres_db",
    database=os.environ.get('POSTGRES_DB'),
    user=os.environ.get('POSTGRES_USER'),
    password=os.environ.get('POSTGRES_PASSWORD')
)

query = "SELECT * FROM reporting_results2020"
df = pd.read_sql_query(query, conn)

/tmp/ipykernel_129/2587181120.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


### Displaying a sample of downloaded data

In [4]:
df.head()

,id,numer_ewidencyjny_system,numer_ewidencyjny_urzad,data_wplywu_wniosku_do_urzedu,nazwa_organu,wojewodztwo_objekt,obiekt_kod_pocztowy,miasto,terc,cecha,...,kubatura,stan,jednostki_numer,obreb_numer,numer_dzialki,numer_arkusza_dzialki,nazwisko_projektanta,imie_projektanta,projektant_numer_uprawnien,projektant_pozostali
0,1,ST-SL-BI/ZGŁOSZENIE/1432/2020,Wb.6743.3.107.2020.KaM,2020-06-04 00:00:00,Starosta Powiatu Bielsko-Biała,śląskie,43-518,Ligota,2402045,,...,,Brak sprzeciwu,240204_5,0005,2395/8,,Świder,Katarzyna,SLK/4131/PWOS/12,
1,2,ST-SL-BI/ZGŁOSZENIE/1432/2020,Wb.6743.3.107.2020.KaM,2020-06-04 00:00:00,Starosta Powiatu Bielsko-Biała,śląskie,43-518,Ligota,2402045,,...,,Brak sprzeciwu,240204_5,0005,2395/5,,Świder,Katarzyna,SLK/4131/PWOS/12,
2,3,ST-SL-RB/ZGŁOSZENIE/6774/2021,AB.6743.915.2021,2021-11-26 00:00:00,Starosta Powiatu Rybnik,śląskie,44-238,Przegędza,2412015,ul.,...,,Brak sprzeciwu,241201_5,0004,1389/6,,Towarek,Grzegorz,SLK/2409/ZOOS/08,
3,4,ST-SL-RB/ZGŁOSZENIE/6774/2021,AB.6743.915.2021,2021-11-26 00:00:00,Starosta Powiatu Rybnik,śląskie,44-238,Przegędza,2412015,ul.,...,,Brak sprzeciwu,241201_5,0004,1727/10,,Towarek,Grzegorz,SLK/2409/ZOOS/08,
4,5,ST-SL-RB/ZGŁOSZENIE/6774/2021,AB.6743.915.2021,2021-11-26 00:00:00,Starosta Powiatu Rybnik,śląskie,44-238,Przegędza,2412015,ul.,...,,Brak sprzeciwu,241201_5,0004,1850/10,,Towarek,Grzegorz,SLK/2409/ZOOS/08,
